<a href="https://colab.research.google.com/github/Neafiol/Tinkoff/blob/master/Rnn/Rnn_sentiment_dz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import numpy as np

# Сентимент анализ 

пишем сами с нуля

<img src="https://github.com/bentrevett/pytorch-sentiment-analysis/raw/bf8cc46e4823ebf9af721b595501ad6231c73632/assets/sentiment1.png">

In [0]:
import torch
from torchtext import data

SEED = 1234

torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize='spacy')
LABEL = data.LabelField(dtype=torch.float)

device = torch.device('cuda:0')

In [3]:
from torchtext import datasets
train_data, test_data = datasets.IMDB.splits(TEXT, LABEL, root="./data")

aclImdb_v1.tar.gz:   0%|          | 0.00/84.1M [00:00<?, ?B/s]

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:01<00:00, 62.2MB/s]


In [4]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of testing examples: {len(test_data)}')

print(vars(train_data.examples[0])['text'])
len(train_data.examples)

Number of training examples: 25000
Number of testing examples: 25000
['Marvelous', 'James', 'Stewart', ',', 'Vera', 'Miles', 'vehicle', '.', 'What', 'makes', 'this', 'historical', 'film', 'dealing', 'with', 'the', 'FBI', 'so', 'good', 'is', 'the', 'family', 'element', 'that', 'is', 'involved', 'during', 'a', '35', 'year', 'career', 'as', 'depicted', 'by', 'Stewart', 'in', 'the', 'film.<br', '/><br', '/>The', 'film', 'shows', 'a', 'history', 'of', 'the', 'great', 'investigatory', 'agency', '.', 'It', 'deals', 'with', 'airplane', 'bomb', 'plots', ',', 'killing', 'off', 'of', 'Indians', 'in', 'Oklahoma', 'for', 'real', 'estate', 'gain', ',', 'fighting', 'organized', 'crime', ',', 'Nazis', 'and', 'Communists', 'in', 'that', 'order', '.', 'The', 'human', 'element', 'is', 'never', 'far', 'behind', 'as', 'Stewart', 'weds', 'Vera', 'Miles', '.', 'They', 'raise', '3', 'children', 'as', 'Miles', "'", 'heart', 'goes', 'out', 'each', 'time', 'Stewart', 'goes', 'out', 'on', 'assignment.<br', '/><br

25000

In [5]:
# Сделаем еще eval
import random

train_data, valid_data = train_data.split(random_state=random.seed(SEED))

# Сделаем словарь
TEXT.build_vocab(train_data, max_size=25000)
LABEL.build_vocab(train_data)

print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

print(TEXT.vocab.itos[:10])
vars(LABEL.vocab)

Unique tokens in TEXT vocabulary: 25002
Unique tokens in LABEL vocabulary: 2
['<unk>', '<pad>', 'the', ',', '.', 'and', 'a', 'of', 'to', 'is']


{'freqs': Counter({'neg': 8810, 'pos': 8690}),
 'itos': ['neg', 'pos'],
 'stoi': defaultdict(<function torchtext.vocab._default_unk_index>,
             {'neg': 0, 'pos': 1}),
 'vectors': None}

In [0]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# собираем батчи так, чтобы в каждом батче были примеры наиболее похожей длины
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size=BATCH_SIZE,
    device=device)

## Делаем модель

<img src="https://github.com/bentrevett/pytorch-sentiment-analysis/raw/bf8cc46e4823ebf9af721b595501ad6231c73632/assets/sentiment7.png" width="450">

* В эмбеддер (emb = [torch.nn.Embedding(num_embeddings, embedding_dim)](https://pytorch.org/docs/stable/nn.html?highlight=embedding#torch.nn.Embedding)) запихиваем тензор размерностью **[sentence length, batch size]**
* Эмбеддер возвращает тензор размерностью **[sentence length, batch size, embedding dim]**
* RNN (torch.nn.RNN(embedding_dim, hidden_dim)) возвращает 2 тензора, *output* размера [sentence length, batch size, hidden dim] и *hidden* размера [1, batch size, hidden dim]

In [0]:
class RNN(nn.Module):
    def __init__(self, batch_size,input_dim, embedding_dim, hidden_dim, output_dim,dropout=0.3):
        super().__init__()
        
        
        self.batch_size = batch_size

        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.basic_rnn = nn.LSTM(self.embedding_dim, self.hidden_dim, bidirectional=True) 
        self.clas = nn.Linear(self.hidden_dim*2, self.output_dim)
        
        self.sm = nn.Softmax(dim = 1)
        self.dropout = nn.Dropout(dropout) 
        
        
    def init_hidden(self):
        # (num_layers, batch_size, n_neurons)
        return (torch.zeros(1, self.batch_size, self.hidden_dim))
      
    def forward (self, text):
      

        #text = [sent len, batch size]
        
        h=self.init_hidden()
#         print('text: ', text.size())
#         print('text: ', text)
        embedded = self.embedding (text)
        
        out, (hidden, c_0) = self.basic_rnn(embedded) 
#         print("after LSTM: ", hidden.size())
      
        result = torch.cat([hidden[-2,:,:],  hidden[-1,:,:]], dim=1)
        
#         print("after concat: ", result.size())
#         print("out: ", out.size())
        
        output=self.dropout(result)
        output=self.clas(output)
#         print("after linear: ", output.size())

        
  
#         print(output.size())
        
        
        output=self.sm(output)
#         print("after Softmax: ", output.size())



        return output.view(-1,self.output_dim)

In [0]:
input_dim = len(TEXT.vocab.freqs)

batch_size=BATCH_SIZE #64
embedding_dim = 100
hidden_dim = 256
output_dim = 2
N_EPHOCS = 100

In [0]:
import torch.optim as optim

# Device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Model instance
model = RNN(batch_size,input_dim, embedding_dim, hidden_dim, output_dim).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

def get_accuracy(logit, target, batch_size):
    ''' Obtain accuracy for training round '''
    corrects = (torch.max(logit, 1)[1].view(target.size()).data == target.data).sum()
    accuracy = 100.0 * corrects/batch_size
    return accuracy.item()
  

In [0]:
for epoch in range(N_EPHOCS):  # loop over the dataset multiple times
    train_running_loss = 0.0
    train_acc = 0.0
    model.train()
    
    # TRAINING ROUND
    for i, data in enumerate(train_iterator):
         # zero the parameter gradients
        optimizer.zero_grad()
        
        # get the inputs
        inputs, labels = data.text, data.label.long()
        
#         print("inputs",inputs.size())
        
        if(inputs.cpu().shape[1]!=batch_size):
#           print(inputs.shape)
          break

        # forward + backward + optimize
        
        out = model(inputs)

        loss = criterion(out, labels)
        
        loss.backward()
        optimizer.step()
        

        train_running_loss += loss.detach().item()
        train_acc += get_accuracy(out, labels, BATCH_SIZE)
#         break
#     break
         
    model.eval()
    print('Epoch:  %d | Loss: %.4f | Train Accuracy: %.2f | loss: %.2f' 
          %(epoch, train_running_loss / i, train_acc/i,loss))

Epoch:  0 | Loss: 0.6353 | Train Accuracy: 62.99 | loss: 0.52
Epoch:  1 | Loss: 0.4903 | Train Accuracy: 81.11 | loss: 0.63
Epoch:  2 | Loss: 0.4565 | Train Accuracy: 84.73 | loss: 0.47
Epoch:  3 | Loss: 0.4433 | Train Accuracy: 86.11 | loss: 0.39
Epoch:  4 | Loss: 0.4305 | Train Accuracy: 87.55 | loss: 0.39
Epoch:  5 | Loss: 0.4487 | Train Accuracy: 85.63 | loss: 0.47
Epoch:  6 | Loss: 0.4337 | Train Accuracy: 87.16 | loss: 0.47
Epoch:  7 | Loss: 0.4366 | Train Accuracy: 86.90 | loss: 0.46
Epoch:  8 | Loss: 0.4281 | Train Accuracy: 87.82 | loss: 0.49
Epoch:  9 | Loss: 0.4130 | Train Accuracy: 89.38 | loss: 0.53
Epoch:  10 | Loss: 0.4326 | Train Accuracy: 87.30 | loss: 0.45
Epoch:  11 | Loss: 0.4383 | Train Accuracy: 86.63 | loss: 0.47
Epoch:  12 | Loss: 0.4303 | Train Accuracy: 87.54 | loss: 0.45
Epoch:  13 | Loss: 0.4421 | Train Accuracy: 86.37 | loss: 0.50
Epoch:  14 | Loss: 0.4161 | Train Accuracy: 89.05 | loss: 0.41
Epoch:  15 | Loss: 0.4447 | Train Accuracy: 86.16 | loss: 0.41
Ep